# Version

* `v12`: Fold4
* `v11`: Fold3
* `v10`: Fold2
* `v08`: Fold1
* `v07`: Fold0


# [Training Notebook](https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class)
* Select `GPU` as the **Accelerator**

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test'
# weights_dir = '/kaggle/input/vinbigdata-cxr-ad-yolov5-14-class-train/yolov5/runs/train/exp/weights/best.pt'
weights_dir = f'/kaggle/input/softnms/best_softnms.pt'

In [ ]:
test_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test.csv')
test_df.head()

# YOLOv5 Stuff

In [ ]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
# shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', './yolov5')

os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Inference

In [ ]:
!python detect.py --weights $weights_dir\
--img 640\
--conf 0.15\
--iou 0.4\
--source $test_dir\
--save-txt --save-conf --exist-ok

# Plot

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*png')
for _ in range(6):
    row = 4
    col = 4
    grid_files = random.sample(files, row*col)
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

# Process Submission

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
#     print('bboxes',bboxes)
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
# image_ids = []
# PredictionStrings = []

# for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
#     image_id = file_path.split('/')[-1].split('.')[0]
#     w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
#     f = open(file_path, 'r')
#     data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
#     data = data[:, [0, 5, 1, 2, 3, 4]]
    
    
#     list_classid_probility = data[:, :2]
#     list_normalized_box = data[:, 2:]
#     filtered_list_classid_probility=[]
#     filtered_list_normalized_box=[]
    
#     for i in range(len(list_classid_probility)):
#         if Hotmask_mid_value(list_classid_probility[i][0] , list_normalized_box[i])>0.1 or Hotmask_w_h_value(list_classid_probility[i][0] , list_normalized_box[i])>0.1 or list_classid_probility[i][0]==14:
#             filtered_list_classid_probility.appendpend(list_classid_probility[i][0])
#             filtered_list_normalized_box.appendpend(list_classid_probility[i][0])
            
#     bboxes = list(np.round(np.concatenate((filtered_list_classid_probility, np.round(yolo2voc(h, w, filtered_list_normalized_box))), axis =1).reshape(-1), 1).astype(str))
#     print('data[:, 2:]',data[:, 2:])
#     for idx in range(len(bboxes)):
#         bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
#     image_ids.append(image_id)
#     PredictionStrings.append(' '.join(bboxes))

In [ ]:
import os 
os.getcwd()

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

dim = 512 #512, 256, 'original'
fold = 4

train_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/train.csv')
train_df.head()

train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.head()



In [ ]:
train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

In [ ]:
list_mid_gaussian_kde=[]
for i in range(14):
    

    train_df_0 = train_df[train_df.class_id==i]
    x_val = train_df_0.x_mid
    y_val = train_df_0.y_mid
    print(len(x_val))

    # Calculate the point density
    xy = np.vstack([x_val,y_val])
    z = gaussian_kde(xy)(xy)

    fig, ax = plt.subplots(figsize = (10, 10))
    # ax.axis('off')
    ax.axis([0,1,1,0])
    ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
    # ax.set_xlabel('x_mid')
    # ax.set_ylabel('y_mid')
    plt.show()

    evalutor=gaussian_kde(xy)
    # evalutor.evaluate([[0.6],[0.4]])
    list_mid_gaussian_kde.append(evalutor)

In [ ]:
list_w_h_gaussian_kde=[]
for i in range(14):
    

    train_df_0 = train_df[train_df.class_id==i]
    x_val = train_df_0.w
    y_val = train_df_0.h
    print(len(x_val))

    # Calculate the point density
    xy = np.vstack([x_val,y_val])
    z = gaussian_kde(xy)(xy)

    fig, ax = plt.subplots(figsize = (10, 10))
    # ax.axis('off')
    ax.axis([0,1,1,0])
    ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
    # ax.set_xlabel('x_mid')
    # ax.set_ylabel('y_mid')
    plt.show()

    evalutor=gaussian_kde(xy)
    # evalutor.evaluate([[0.6],[0.4]])
    list_w_h_gaussian_kde.append(evalutor)

In [ ]:
# https://www.pythonheidong.com/blog/article/436765/a0facf9464d9337dc1eb/
list_area_gaussian_kde=[]
for i in range(14):
    train_df_0 = train_df[train_df.class_id==i]
    data = train_df_0.w

    density = gaussian_kde(data)
    xs = np.linspace(0,1,200)
    plt.plot(xs,density(xs))
    plt.show()
    list_area_gaussian_kde.append(density)

In [ ]:
print(density.evaluate(0.4))

In [ ]:
def Hotmask_mid_value(class_id,normalized_xmid_ymid_w_h):
    evaluator = list_mid_gaussian_kde[int(class_id)]
    value = evaluator.evaluate([[normalized_xmid_ymid_w_h[0]],[normalized_xmid_ymid_w_h[1]]])
    
    return value

def Hotmask_w_h_value(class_id,normalized_xmid_ymid_w_h):
    evaluator = list_w_h_gaussian_kde[int(class_id)]
    value = evaluator.evaluate([[normalized_xmid_ymid_w_h[2]],[normalized_xmid_ymid_w_h[3]]])
    
    return value

def Hotmask_area_value(class_id,normalized_xmid_ymid_w_h):
    evaluator = list_area_gaussian_kde[int(class_id)]
    value = evaluator.evaluate(normalized_xmid_ymid_w_h[2]*normalized_xmid_ymid_w_h[3])
    
    return value

In [ ]:
image_ids = []
PredictionStrings = []
sum_delte_on=0

# threshold=0.001
threshold_mid=0.001
threshold_w_h=0.001
threshold_area=0.001

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    delte_on=0
    
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
#     print('data',data.type)
    
    list_classid_probility = data[:, :2]
    list_normalized_box = data[:, 2:]
    delte_sign=[]
#     filtered_list_classid_probility=[]
#     filtered_list_normalized_box=[]
    
#     print('data[:, :2]',data[:, :2]) 
    
    for i in range(len(list_classid_probility)):
        if (Hotmask_mid_value(list_classid_probility[i][0] , list_normalized_box[i])>threshold_mid and Hotmask_w_h_value(list_classid_probility[i][0] , list_normalized_box[i])>threshold_w_h and Hotmask_area_value(list_classid_probility[i][0] , list_normalized_box[i])>threshold_area) or list_classid_probility[i][0]==14:
            delte_sign.append(True)
        else:
            delte_sign.append(False)
            delte_on+=1
            
#             filtered_list_classid_probility.append(list_classid_probility[i][0])
#             filtered_list_normalized_box.append(list_classid_probility[i].tolist())
            
#     prin**t('filtered_list_normalized_box',filtered_list_normalized_box) 
    
    print('delte_sign',delte_sign) 
    bboxes = list(np.round(np.concatenate((data[:, :2][delte_sign], np.round(yolo2voc(h, w, data[:, 2:][delte_sign]))), axis =1).reshape(-1), 1).astype(str))
#     bboxes = np.array(list(np.round(np.concatenate((filtered_list_classid_probility, np.round(yolo2voc(h, w, filtered_list_normalized_box))), axis =1).reshape(-1), 1).astype(str)),dtype=np.float32)
#     print('data[:, 2:]',data[:, 2:])
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))
    
    if delte_on!=0:
        sum_delte_on+=1

In [ ]:
sum_delte_on

In [ ]:
# image_ids = []
# PredictionStrings = []

# for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
#     image_id = file_path.split('/')[-1].split('.')[0]
#     w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
#     f = open(file_path, 'r')
#     data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
#     data = data[:, [0, 5, 1, 2, 3, 4]]
#     bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
# #     print('data[:, :2]',data[:, :2].type)
#     for idx in range(len(bboxes)):
#         bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        
#     print('after bboxes',bboxes)
#     image_ids.append(image_id)
#     PredictionStrings.append(' '.join(bboxes))

In [ ]:
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df = sub_df[['image_id', 'PredictionString']]

sub_df.loc[sub_df['PredictionString'] =='','PredictionString']="14 1 0 0 1 1"
sub_df.to_csv('/kaggle/working/submission_softnms.csv',index = False)
sub_df.tail()

In [ ]:
# sub_df.tail()

In [ ]:
# test_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test.csv')
# test_df.head()

In [ ]:
# imagepaths = test_df['image_id'].unique()
# imagepaths

In [ ]:
# for i, name in tqdm(enumerate(imagepaths)):
#     path=os.path.join('../input/vinbigdata-original-image-dataset/vinbigdata/train',name+'.jpg')
#     img_array  = cv2.imread(path)
#     print('name',name)
#     print('shape',img_array.shape)

In [ ]:
# train_data = np.array(sub_df)#np.ndarray()
# train_x_list=train_data.tolist()#list
# print(train_x_list)
# print(type(train_x_list))

In [ ]:
# shutil.rmtree('/kaggle/working/yolov5')